In [1]:
%matplotlib inline 
import itertools
from bs4 import BeautifulSoup
import urllib2
import requests
import pandas as pd
import re
import time
import numpy as np
import json
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import matplotlib.pyplot as plt
from pyquery import PyQuery as pq
dropbox = "C:\Users\mkkes_000\Dropbox\Indiastuff\OutputTables"

In [2]:
list_elections = ["andhra2014","arunachal2014","bihar2015","chhattisgarh2013","delhi2015","gujarat2012","haryana2014","hp2012","jk2014",
              "jharkhand2014","karnataka2013","kerala2011","mp2013","maharashtra2014","manipur2012","odisha2014","pb2012",
              "rajasthan2013","sikkim2014","tamilnadu2011","telangana2014","tripura2013","utt2012","up2012","westbengal2011"]

In [7]:
#For each election we take the district name and the constituency names.
# We want to create a tuple for each constituency
# of structure (name, district, id_number)

def only_highest(l):
    list_highest = []
    for i in range(0,len(l)-1):
        if l[i+1]>l[i]:
            list_highest = list_highest
        else:
            list_highest.append(l[i])
    return list_highest

def constituency_tuple(e):
    long_link = "http://myneta.info/"+e+"/"
    source=requests.get(long_link)
    time.sleep(1)
    tree= BeautifulSoup(source.text,"html.parser")
    names_districts = [x.get_text().strip().title() for x in tree.findAll("h5", { "class" : "title" })]
    list_const = tree.findAll("div", { "class" : "items" })[1:]
    names_constituencies = [x.get_text().split(":")[1].strip().title().split("*")[0] for x in list_const]
    link_constit = [x.find("a").get("href") for x in tree.findAll("div", { "class" : "items" })[1:]]
    id_constit = [x.split("=")[2] for x in link_constit]
    # Only taking the first election
    n_constituencies = only_highest([int(x.get_text().split(":")[0].split("(")[0])
                                 for x in tree.findAll("div", { "class" : "items" })[1:]])
    list_districts = list(itertools.chain(
                    *[[names_districts[i]]*n_constituencies[i] for i in range(0,len(names_districts)-1)]))
    return zip(names_constituencies,list_districts,id_constit)

In [8]:
%%time
#Creates a dictionary state:[all constituencies] 
#where all constituency is a tuple (constituency,district,id of constituency)
all_elections = {el:constituency_tuple(el) for el in list_elections}

Wall time: 49.2 s


In [ ]:
#just building dictionaries to store info more efficiently
def store_dict(el):
    dtuple = all_elections[el]
    constituency_district =  {l[2]:l[1] for l in dtuple}
    return constituency_district

def store_dict2(el):
    dtuple = all_elections[el]
    constituency_code =  {l[2]:l[0] for l in dtuple}
    return constituency_code

def store_constituencies(el):
    dtuple = all_elections[el]
    allcodes =  [l[2] for l in dtuple]
    return allcodes

election_constituencies = {}
allconst_dist = {}
allconst_codes = {}
for el in list_elections:
    allconst_dist[el] = store_dict(el)
    allconst_codes[el] = store_dict2(el)
    election_constituencies[el] = store_constituencies(el)

with open("tempdata/const_district",'w') as js_file:
    json.dump(allconst_dist,js_file)
with open("tempdata/const_codes",'w') as js_file:
    json.dump(allconst_codes,js_file)

In [ ]:

def cand_info(line):
    all_info = line.findAll("td")
    name = all_info[0].get_text()
    cand_link = all_info[0].find("a").get("href")
    cand_id = cand_link.split("=")[1]
    try:
        age = int(all_info[1].get_text())
    except TypeError:
        age = np.nan
    sex = all_info[2].get_text()
    party = all_info[3].get_text()
    serious_crim_yn = all_info[4].get_text()
    try:
        crim = int(all_info[5].get_text())
        serious_crim_num = int(all_info[6].get_text())
    except ValueError:
        crim = np.nan
        serious_crim_num = np.nan
    if all_info[7].get_text()=="":
        crim_types = []
    else:
        crim_types = [x.get_text() for x in all_info[7].findAll("span")]
    educ = all_info[8].get_text()
    list_assets = [int(all_info[i].get_text().split("~")[0].replace(',', '')) for i in [9,10,11,12]]
    #mov_assets = all_info[9].get_text().split("~")[0]
    #immov_assets = all_info[10].get_text().split("~")[0]
    #total_assets = all_info[11].get_text().split("~")[0]
    #total_liab = all_info[12].get_text().split("~")[0]
    pan = all_info[13].get_text()
    list_all = [cand_link,cand_id,name,age,sex,party,serious_crim_yn,crim,serious_crim_num,crim_types,educ]+list_assets+[pan]
    return list_all

In [ ]:
# Creates a dictionary of dictionaries in the format:
# {election:{constituency_id:detailed constituency page}}
def get_page(c,election):
    # Check if URL has already been visited.
    url_error = []
    if (c not in urlcache) or (urlcache[c]==1) or (urlcache[c]==2):
        time.sleep(.5)
        steps = len(urlcache)
        if 100*int(steps/100)==steps:
            print steps # This counter tells us how many links were downloaded at every 100 mark
        # try/except blocks are used whenever the code could generate an exception (e.g. division by zero).
        # In this case we don't know if the page really exists, or even if it does, if we'll be able to reach it.
        try:
            link_disc = "http://myneta.info/"+election+"/comparisonchart.php?constituency_id="+c
            c_page=requests.get(link_disc)
            if c_page.status_code == 200:
                urlcache[c] = c_page.text
            else:
                urlcache[c] = 1
        except:
            urlcache[c] = 2
            url_error.append(c)
            print c
    return urlcache[c]

def get_info_constituency(election):
    const_list = election_constituencies[election]
    print "\n", election, " of length", len(const_list), ":",
    dict_store = {}
    for c in const_list:
        link_disc = "http://myneta.info/"+election+"/comparisonchart.php?constituency_id="+c
        time.sleep(.5)
        c_page=requests.get(link_disc)
        dict_store[c] = c_page
        if len(dict_store)%100==0 or len(dict_store)==len(const_list):
            print len(dict_store),
    return dict_store

In [ ]:
%%time
# The structure is the following:
# List of dictionaries {state:{constituency:page}}
for el in list_elections:
    if el in dict_all_pages:
        if len(dict_all_pages[el]) == len(election_constituencies[el]):
            print "done for:", el, "length:", len(dict_all_pages[el])
        else:
            print "starting again for ", el
    else:
        dict_all_pages[el] = get_info_constituency(el) #careful, very long call

In [ ]:
#The all_const function is at the constituency level
# Taking a constituency in a given state, it spits out a list of candidates with their info
def all_candidates(c,election):
    dict_c = dict_all_pages[election]
    const_page = dict_c[c]
    tree = BeautifulSoup(const_page.text,"html.parser")
    cand_lines = tree.findAll("tr")[1:]
    constituency = allconst_codes[election][c]
    district = allconst_dist[election][c]
    titles = ["election","c","constituency","district","link","id"]+[x.get_text() for x in tree.findAll("th")]
    info_candidates = [dict(zip(titles,[election,c,constituency,district]+cand_info(r))) for r in cand_lines]
    return info_candidates
def state_mapper(el):
    all_cand_state = [all_candidates(c_line[2],el) for c_line in all_elections[el]]
    return all_cand_state
## Takes one of the elements of the all_election_final dictionary
## And spits out a dataframe with all the columns. We can then stack them
## at a later stage
def reducer(el):
    list_const = all_elections_final[el]
    large_list = []
    for cons in list_const:
        large_list.extend(cons)
    df = pd.DataFrame(large_list)
    return df

In [ ]:
%%time
all_elections_dict = {}
for e in list_elections:
    all_elections_dict[e]=state_mapper(e)
    print e,
    all_elections_final = all_elections_dict.copy()
df_all = pd.DataFrame()
for el in list_elections:
    df = reducer(el)
    df_all = df_all.append(df)
df_fin = df_all.copy()

In [ ]:
%%time
## finding winners ##
winners_dict = {}
for el in list_elections:
    link_winners = "http://myneta.info/"+el+"/index.php?action=summary&subAction=winner_analyzed&sort=candidate#summary"
    wins = requests.get(link_winners)
    time.sleep(1)
    tree = BeautifulSoup(wins.text,"html.parser")
    tables = tree.findAll("table")
    list_ids = [x.find("a").get("href").split("=")[1] for x in tables[2].findAll("tr")[2:]]
    winners_dict[el]=list_ids

In [ ]:
def comparison_clean(line,el):
    id2 = line.find("a").get("href").split("=")[3].split("&")[0] #id2 is the latest
    # We are going to match it to the df and add the new id + old info
    id1 = line.find("a").get("href").split("=")[4]
    elements = [x.get_text() for x in line.findAll("td")]
    name = elements[1].split("(")[0].strip()
    new_assets = elements[2].split(u'\xa0')[0].replace(",","")
    old_assets = elements[3].split(u'\xa0')[0].replace(",","")
    increase = elements[4].split(u'\xa0')[0].replace(",","")
    try:
        increase_pct = float(elements[5].replace('%',''))
    except ValueError:
        increase_pct = elements[5].replace('%','')
    list_repeat = [el,id2,id1,name,new_assets,old_assets,increase,increase_pct]
    titles = ['election','new_id','old_id','name2','assets new','assets old','increase','pct change']
    dict_repeat = dict(zip(titles,list_repeat))
    return dict_repeat

In [ ]:
%%time
df_repeat = pd.DataFrame()
for el in list_elections:
    link_repeat = "http://myneta.info/"+el+"/index.php?action=recontestAssetsComparison"
    page = requests.get(link_repeat)
    tree = BeautifulSoup(page.text,"html.parser")
    tables = tree.findAll("table")
    all_lines = tables[2].findAll("tr")[2:]
    info_repeat = [comparison_clean(l,el) for l in all_lines if len(l)==7]
    df_state = pd.DataFrame(info_repeat)
    df_repeat = df_repeat.append(df_state)


In [ ]:
for el,list_id in winners_dict.items():
    df_fin['win'][df_fin.election==el]= df_fin.apply(lambda row: (row['election']==el)==True and (row['id'] in list_id)==True, axis=1)

In [ ]:
df_fin.to_csv("tempdata\df_all_regional_elections.csv")

In [ ]:
def url_tsform(el,c_id):
    url_link = "http://myneta.info/%s/candidate.php?candidate_id=%s" %(el,c_id)
    return url_link
def url_split(url_link):
    return url_link.split("/")[3], url_link.split("=")[1]
def get_page(el,c_id):
    # Check if URL has already been visited.
    url = (el,c_id)
    url_error = []
    if (url not in urlcache) or (urlcache[url]==1) or (urlcache[url]==2):
        time.sleep(.5)
        try:
            r = requests.get(url_tsform(el,c_id))
            if r.status_code == 200:
                urlcache[url] = r.text
            else:
                urlcache[url] = 1
        except:
            urlcache[url] = 2
            url_error.append(url)
            print "error with:", url
    return urlcache[url]

In [ ]:
#Creates a giant dictionary
#dict_id2 = {}
steps = len(dict_id2)
print "initial length: ", steps
for row in df_fin.itertuples():
    if (row[19],row[20]) in dict_id2:
        pass
    else:
        dict_id2[(row[19],row[20])] = get_page(row[19],row[20])
        steps = len(dict_id2)
        if steps % 1000 ==0:
            print steps,

In [ ]:
dict_clean = {url_tsform(k[0],k[1]):v for k,v in dict_id2.iteritems() if v!=2}
print len(dict_clean), len(dict_id2)

In [ ]:
with open('tempdata/largedict.json', 'w') as f:
     json.dump(dict_clean, f)

In [ ]:
# We now create complete information file on each 37000 candidates 
# On assets, literacy, income (when available) and criminality
# As well as whether they have repeated information

In [2]:
#Re open the dict
with open('tempdata/largedict.json', 'r') as f:
     dict_start = json.load(f)

In [3]:
# Get the assets for each candidate #
def movable_assets(info,url):
    soup=BeautifulSoup(info[url],"html.parser")
    table = soup.find("a", attrs={"name": "movable_assets"}).find("table")
    madict={}
    madict["url"]=url
    
    if url[19:21]=="lo":
        madict['year']=url[len('http://myneta.info/loksabha'): 4 + len('http://myneta.info/loksabha')]
    else:
        madict['year']=url[len('http://myneta.info/ls'): 4 + len('http://myneta.info/ls')]
   
    for r in table.findAll('td'):
        try:
            if r.get_text()=="i":
                madict["Cash"] = r.findNext('b').get_text().strip().encode('ascii', 'ignore').replace("Rs","")

            if r.get_text()=="ii":
                madict["Deposits in Banks, Financial Institutions and Non-Banking Financial Companies"] = r.findNext('b').get_text().encode('ascii', 'ignore').replace("Rs","").strip()

            if r.get_text()=="iii":
                madict["Bonds, Debentures and Shares in companies"] = r.findNext('b').get_text().encode('ascii', 'ignore').replace("Rs","").strip()
                             # use different tag and attribute to pick up text. Limit specifies # of iteration for findall method
            if r.get_text()=="iv":
                madict["NSS, Postal Savings etc"] = r.findAllNext('b',limit=4)[1].get_text().encode('ascii', 'ignore').replace("Rs","").strip()
                madict["LIC or other insurance Policies"] = r.findAllNext('b',limit=4)[3].get_text().encode('ascii', 'ignore').replace("Rs","").strip()
           
            
            if url[19:25]=="ls2014":
                
                if r.get_text()=="v":
                    madict["Personal loans/advance given"] = r.findNext('b').get_text().encode('ascii', 'ignore').replace("Rs","").strip()

                if r.get_text()=="vi":
                    madict["Motor Vehicles (details of make, etc.)"] = r.findNext('b').get_text().encode('ascii', 'ignore').replace("Rs","").strip()

                if r.get_text()=="vii":
                    madict["Jewellery (give details weight value)"] = r.findNext('b').get_text().encode('ascii', 'ignore').replace("Rs","").strip()

                if r.get_text()=="viii":
                    madict["Other assets, such as values of claims / interests"] = r.findNext('b').get_text().encode('ascii', 'ignore').replace("Rs","").strip()

                if r.get_text()=="Gross Total Value (as per Affidavit)":
                    madict["Gross Total Value (as per Affidavit)"] = r.findAllNext('b',limit=1)[0].get_text().encode('ascii', 'ignore').replace("Rs","").strip()

                if r.get_text()=="Totals (Calculated as Sum of Values)":
                    madict["Totals (Calculated as Sum of Values)"] = r.findAllNext('b',limit=6)[5].get_text().encode('ascii', 'ignore').replace("Rs","").strip()
                    
                madict["TOTAL_MOVABLE_ASSESTS"]=  int(madict["Totals (Calculated as Sum of Values)"].replace(',','') )
      
            else:
                
                if r.get_text()=="v":
                    madict["Motor Vehicles (details of make, etc.)"] = r.findNext('b').get_text().encode('ascii', 'ignore').replace("Rs","").strip()

                if r.get_text()=="vi":
                    madict["Jewellery (give details weight value)"] = r.findNext('b').get_text().encode('ascii', 'ignore').replace("Rs","").strip()

                if r.get_text()=="vii":
                    madict["Other assets, such as values of claims / interests"] = r.findNext('b').get_text().encode('ascii', 'ignore').replace("Rs","").strip()

                if r.get_text()=="Totals":
                    madict["Totals (Calculated as Sum of Values)"] = r.findAllNext('b', limit=6)[5].get_text().encode('ascii', 'ignore').replace("Rs","").strip()
        
                if madict["LIC or other insurance Policies"]=="Nil":
                    madict["TOTAL_MOVABLE_ASSETS"]=  int(madict["Totals (Calculated as Sum of Values)"].replace(',',''))
                else:
                    madict["TOTAL_MOVABLE_ASSETS"]=  int(madict["Totals (Calculated as Sum of Values)"].replace(',','')) + int(madict["LIC or other insurance Policies"].replace(',',''))
        
        except:
            pass    
        

    return madict

def immovable_assets(info,url):
    soup=BeautifulSoup(info[url],"html.parser")
    table = soup.findAll("table",attrs={"class":"table1"})[2]
    immadict={}
    immadict["url"]=url
    
    for r in table.findAll('td'):
        try:
            if r.get_text()=="i":
                immadict["Agricultural Land"] = r.findNext('b').get_text().strip().encode('ascii', 'ignore').replace("Rs","")

            if r.get_text()=="ii":
                immadict["Non Agricultural Land"] = r.findNext('b').get_text().strip().encode('ascii', 'ignore').replace("Rs","")
           
            if r.get_text()=="iii":
                immadict["Commercial Buildings"] = r.findNext('b').get_text().strip().encode('ascii', 'ignore').replace("Rs","")

            if r.get_text()=="iv":
                immadict["Residential Buildings"] = r.findNext('b').get_text().strip().encode('ascii', 'ignore').replace("Rs","")

            if r.get_text()=="v":
                immadict["Others"] = r.findNext('b').get_text().strip().encode('ascii', 'ignore').replace("Rs","")
            
            if url[19:25]=="ls2014":
                
                if r.get_text()=="Total Current Market Value of (i) to (v) (as per Affidavit)":
                    immadict["Total Current Market Value of (i) to (v) (as per Affidavit)"] = r.findAllNext('b',limit=1)[0].get_text().strip().encode('ascii', 'ignore').replace("Rs","")

                if r.get_text()=="Totals Calculated":
                    immadict["TOTAL_IMMOVABLE_ASSETS"] = r.findAllNext('b',limit=6)[5].get_text().strip().encode('ascii', 'ignore').replace("Rs","")
            else:
                if r.get_text()=="Totals":
                    immadict["TOTAL_IMMOVABLE_ASSETS"] = r.findAllNext('b',limit=6)[5].get_text().strip().encode('ascii', 'ignore').replace("Rs","")
                
        except:
            pass
    
    return immadict
def cleaner(d):
    for key in d.keys():
        if d[key]=='Nil' or d[key]=='':
            d[key]==np.nan
        else:
            try:
                d[key]=float(d[key].replace(',',''))
            except:
                pass
    return d

In [6]:
%%time
movassets={}
for url in dict_start.keys():
    try:
        massetdict=cleaner(movable_assets(dict_start,url))
        movassets[url] = massetdict
        if len(movassets)%1000==0:
            print len(movassets),
    except TypeError: 
        print "error: ",url

 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000Wall time: 1h 12min 23s



In [17]:
%%time
immovassets={}
for url in dict_start.keys():
    try:
        immov_assetdict=cleaner(immovable_assets(dict_start,url))
        immovassets[url] = immov_assetdict
        if len(immovassets)%1000==0:
            print len(immovassets),
    except TypeError: 
        print "error: ",url

1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000Wall time: 48min 19s



In [19]:
test_key = immovassets.keys()[12]
print "Immov assets:", immovassets[test_key], "\nMovables: ", movassets[test_key]

Immov assets: {'Residential Buildings': 500000.0, 'url': u'http://myneta.info/westbengal2011/candidate.php?candidate_id=35', 'Others': 'Nil', 'Commercial Buildings': 'Nil', 'year': u'stbe', 'Non Agricultural Land': 200000.0, 'Agricultural Land': 'Nil'} 
Movables:  {'Jewellery (give details weight value)': 'Nil', 'url': u'http://myneta.info/westbengal2011/candidate.php?candidate_id=35', 'NSS, Postal Savings etc': 'Nil', 'year': u'stbe', 'Cash': 20000.0, 'Other assets, such as values of claims / interests': 'Nil', 'LIC or other insurance Policies': 'Nil', 'Bonds, Debentures and Shares in companies': 'Nil', 'Deposits in Banks, Financial Institutions and Non-Banking Financial Companies': 17223.0, 'Motor Vehicles (details of make, etc.)': 'Nil'}


In [22]:
with open('tempdata/mov_assets_state.json','w') as f:
    json.dump(movassets,f)
with open('tempdata/immov_assets_state.json','w') as f:
    json.dump(immovassets,f)   

df_mov_assets = pd.DataFrame.from_dict(movassets, orient = 'index')
df_immov_assets = pd.DataFrame.from_dict(immovassets, orient = 'index')
df_assets = pd.merge(df_mov_assets,df_immov_assets, on = 'url')
df_assets.drop(['year_x','year_y'], inplace=True, axis=1)

In [23]:
df_assets.head()

,Jewellery (give details weight value),url,"NSS, Postal Savings etc",Cash,"Other assets, such as values of claims / interests",LIC or other insurance Policies,"Bonds, Debentures and Shares in companies","Deposits in Banks, Financial Institutions and Non-Banking Financial Companies","Motor Vehicles (details of make, etc.)",Residential Buildings,Others,Commercial Buildings,Non Agricultural Land,Agricultural Land
0,210000,http://myneta.info/andhra2014/candidate.php?ca...,Nil,13000,210000,990000,Nil,11000,Nil,Nil,Nil,Nil,800000,Nil
1,Nil,http://myneta.info/andhra2014/candidate.php?ca...,Nil,15000,150000,Nil,Nil,1500,Nil,1200000,Nil,Nil,Nil,Nil
2,0,http://myneta.info/andhra2014/candidate.php?ca...,Nil,20000,Nil,Nil,Nil,0,Nil,110000,Nil,Nil,Nil,0
3,54000,http://myneta.info/andhra2014/candidate.php?ca...,Nil,15000,420000,168788,Nil,27130,Nil,Nil,Nil,Nil,Nil,Nil
4,Nil,http://myneta.info/andhra2014/candidate.php?ca...,Nil,25000,90000,Nil,Nil,1000,Nil,200000,Nil,Nil,Nil,Nil


In [25]:
##Let's now obtain all the criminal data #
def get_crimdata2(year, page_text, url):
    
    crim_list = []
    crim_det_list = []
    success = 0
    crim_dict = {'year_election': year, 'url': url}
    header = ['serial_no','IPC_sections','Other']
    
    for h3 in pq(page_text)("h3"):
        #print h3.text
        if h3.text == "Cases where charges framed":
            framed = pq(h3).next('table')
            crim_dict['type'] = 'Framed'
            for i,tr in enumerate(pq(framed)('tr')):
                crim_dict_copy = crim_dict.copy()
                if i > 0:
                    for j,child in enumerate(tr.iterchildren()):
                        if child.text != '---------':
                            crim_dict_copy[header[j]] = child.text
                            success = 1
                    if success == 1:
                        crim_list.append(crim_dict_copy)
                        success = 0
        if h3.text == "Cases where Cognizance taken":
            framed = pq(h3).next('table')
            crim_dict['type'] = 'Cogniz'
            for i,tr in enumerate(pq(framed)('tr')):
                crim_dict_copy = crim_dict.copy()
                if i > 0:
                    for j,child in enumerate(tr.iterchildren()):
                        if child.text != '---------':
                            crim_dict_copy[header[j]] = child.text
                            success = 1
                    if success == 1:
                        crim_list.append(crim_dict_copy)
                        success = 0
        if h3.text == "Cases where convicted":
            framed = pq(h3).next('table')
            crim_dict['type'] = 'Convict'
            for i,tr in enumerate(pq(framed)('tr')):
                crim_dict_copy = crim_dict.copy()
                if i > 0:
                    for j,child in enumerate(tr.iterchildren()):
                        if child.text != '---------':
                            crim_dict_copy[header[j]] = child.text
                            success = 1
                    if success == 1:
                        crim_list.append(crim_dict_copy)
                        success = 0
        
        if h3.text == "Brief Details of IPCs":
            crim_det_dict = {'url': url}
            parent = pq(h3).parent()
            details = parent.text().split(') ')
            
            for x in details:
                if len(x)>0:
                    crim_det_copy = crim_det_dict.copy()
                    crim_det_copy['detail'] = x
                    crim_det_list.append(crim_det_copy)
        
    return crim_det_list, crim_list

In [27]:
%%time
crimdataset = []
crimdetailset = []
for i,urls in enumerate(dict_start.keys()):
    page_data = dict_start[urls]
    if (i%1000.0) == 0.0:
        print i,
    
    election  = urls.split("/")[3]
    details, dataset = get_crimdata2(election,page_data,urls)
    crimdataset = crimdataset + dataset
    crimdetailset = crimdetailset + details

with open("tempdata/dic_crim_state.json",'w') as f:
    json.dump(crimdataset,f)
with open("tempdata/dic_crim_details_state.json",'w') as f:
    json.dump(crimdetailset,f)
    
criminal_df = pd.DataFrame(crimdataset)
crim_details_df = pd.DataFrame(crimdetailset)
criminal_df.count()

 0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000Wall time: 6min 42s



In [28]:
cleaner = lambda e: int(re.findall('\d+', e.replace(',', '').split(" ~ ")[0])[0])
income_cols = ["Relation","PAN","Year","Income"]

def income_table(link):
    page_candidate = dict_start[link]
    c_soup = BeautifulSoup(page_candidate,"html.parser")
    table_titles =[x.get_text().strip() for x in c_soup.findAll("h3")]
    tables = [x.find_next() for x in c_soup.findAll("h3")]
    dict_tab = dict(zip(table_titles,tables))
    income_tab = dict_tab['Details of PAN and status of Income Tax return']
    income_rows = income_tab.find_all("tr")
    dict_income = {}
    df_inc = pd.DataFrame([])
    if income_cols==[]:
        dict_income = {'HH':{"Year":np.nan,"PAN":"N","Relation":np.nan,"Income":np.nan}}
    else:
        for r in income_rows[1:]:
            list_items = [x.get_text() for x in r.findAll("td")]
            if len(list_items)==4 and list_items[3]!="Nil":
                list_items[3] = cleaner(list_items[3])
            if len(list_items)==4 and list_items[3]=="Nil":
                list_items[3] = 0
            dict_income[list_items[0]] = dict(zip(income_cols,list_items))
        df_inc = df_inc.from_dict(dict_income,orient = "index")
    try:
        df_inc = df_inc[df_inc.PAN=="Y"]
        HHinc = np.sum(df_inc['Income'])
        HHDeclarations = np.count_nonzero(df_inc['PAN'])
        self_income = dict_income['self']['Income']
        self_declare = dict_income['self']['PAN']
    except AttributeError:
        df_inc=df_inc
        HHinc = np.nan
        HHDeclarations = 0
        self_income = np.nan
        self_declare = np.nan
    newdict = {'self_inc':self_income,'self_declare':self_declare,'HHinc':HHinc,"HHDeclarations":HHDeclarations}
    return newdict

In [31]:
%%time
counterror = 0
dict_allinc = {}
for k,url in enumerate(dict_start.keys()):
    try:
        dict_allinc[url] = income_table(url)
    except TypeError:
        counterror = counterror+1
        print "Error with this page: ", url
    except KeyError:
        counterror = counterror+1
        print "Error with this page: ", url
    if k%1000==0:
        print k,
print "\n Number of errors: ", counterror


0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 3400 3500 3600 3700 3800 3900 4000 4100 4200 4300 4400 4500 4600 4700 4800 4900 5000 5100 5200 5300 5400 5500 5600 5700 5800 5900 6000 6100 6200 6300 6400 6500 6600 6700 6800 6900 7000 7100 7200 7300 7400 7500 7600 7700 7800 7900 8000 8100 8200 8300 8400 8500 8600 8700 8800 8900 9000 9100 9200 9300 9400 9500 9600 9700 9800 9900 10000 10100 10200 10300 10400 10500 10600 10700 10800 10900 11000 11100 11200 11300 11400 11500 11600 11700 11800 11900 12000 12100 12200 12300 12400 12500 12600 12700 12800 12900 13000 13100 13200 13300 13400 13500 13600 13700 13800 13900 14000 14100 14200 14300 14400 14500 14600 14700 14800 14900 15000 15100 15200 15300 15400 15500 15600 15700 15800 15900 16000 16100 16200 16300 16400 16500 16600 16700 16800 16900 17000 17100 17200 17300 17400 17500 17600 17700 17800 17900 18000 18100 18200 18300 18400 18

In [38]:
df_income = pd.DataFrame.from_dict(dict_allinc, orient = 'index')
df_income['link'] = df_income.index

In [ ]:
### Now onto the complete info on previous election ###
ROOT_LINK = "http://myneta.info"
def get_otherelec_link(page_text, url):
    for a in pq(page_text)("a"):
        if a.text == "Click here for more details":
            other_elec_link = pq(a).attr.href
            return ROOT_LINK+other_elec_link
    return False

def get_otherelec_data(otherelec_link):
    
    otherelec_dict = {'common_link': otherelec_link}
    
    html = requests.get(otherelec_link)
    doc = pq(html.content)
    
    columns = []
    all_dicts = []
    add = 0
    trs = doc('tr')
    for tr in trs:
        elec_dict = otherelec_dict.copy()
        for th in pq(tr)('th'):
            columns.append(pq(th).text().replace(" ","_"))
            add = 0
        for i,td in enumerate(pq(tr)('td')):
            a = pq(td)('a')
            if a:
                elec_dict['elec_link'] = ROOT_LINK+a.attr.href
                elec_dict[columns[i]] = a.text()
            else:
                try:
                    if pq(td)('span') and i < 6:
                        elec_dict[columns[i]] = pq(td)('span').text()
                    else:
                        elec_dict[columns[i]] = str(pq(td).contents()[0]).encode('utf-8').strip().replace(',','')
                except:
                    print ""
                    print "Skipping col %s for %s" % (columns[i], elec_dict['common_link'])
            add = 1
            
        if add == 1:
            all_dicts.append(elec_dict)
    
    return all_dicts

In [ ]:
def get_all_elec():
    all_elec_data = []   
    counter = 0.0
    for key, val in dict_start.iteritems():
        thelink = get_otherelec_link(val,key)
        counter += 1
        if counter%1000 == 0.0:
            print ".",
        if thelink:
            all_elec_data = all_elec_data + get_otherelec_data(thelink)
    
    df = pd.DataFrame(all_elec_data)
    return df.drop_duplicates()

all_elecs_df = get_all_elec()
all_elecs_df.to_csv("tempdata/all_elecs_df_state.csv",encodig='utf-8')

In [39]:
### Now let's merge everything and obtain a nice dataframe with:
# Basic info (from the constituency page)
# Asset info and criminal info as well as income info when available (from the candidate page)
# Info about the past (from the "previous election" table
#                      in the candidate page and the information in each of those links)

df_main = pd.read_csv("tempdata\df_all_regional_elections.csv")
df_main = df_main.merge(criminal_df, left_on = 'link', right_on = 'url', how = 'left')
df_main = df_main.merge(df_assets, left_on = 'link', right_on = 'url', how = 'left')
df_main = df_main.merge(df_income, on='link', how = 'left')
#df_main = df_main.merge(all_elecs_df, left_on = 'link', right_on = 'url', how = 'left')

In [56]:
df_states2014 = df_main.drop_duplicates(['Name','consistuency'])
df_states2014.to_csv('tempdata/states2014_all',encoding = 'utf-8')

In [57]:
print df_states2014.shape

(37255, 46)
